In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install hopsworks

In [1]:
from sentence_transformers import SentenceTransformer,util

In [2]:
# Loading a text file and splitting it into sentences.

def read_file(input_file_path):
  sentences = []

  with open(input_file_path, 'r') as file:
      content = file.read()
      sentences = content.split('.')
      return sentences

input_file_path = "/content/hydrogen.txt"
sentences = read_file(input_file_path)

In [3]:
# Loading the sentence transformer model and creating sentence embedding for the text
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

In [ ]:
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print()


In [6]:
import hopsworks
import pandas as pd

In [7]:
# Once this cell is run, the user needs to enter the hopsworks API key in the prompt provided.
hopsworks_project = hopsworks.login()
fs = hopsworks_project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/44182
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
# Creating the dataframe which is to be uploaded to the feature group
df = pd.DataFrame(list(zip(sentences, embeddings)),
               columns =['Sentence', 'Embeddding'])
df['p_key'] = [i for i in range(1,len(sentences)+1)]

In [11]:
# Create a feature group
# Provide a name, version, description and the primary key column
sentence_embedding_fg = fs.get_or_create_feature_group(
		name="sentence_embedding",
    version=1,
    description="Embedding data for hydrogen",
    primary_key=['p_key'],
    online_enabled=True)

In [12]:
# Insert the created dataframe to the recently created feature store.
# We will get to know the status of the insertion via the link provided
sentence_embedding_fg.insert(df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/44182/fs/44101/fg/95661


Uploading Dataframe: 0.00% |          | Rows 0/23 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: sentence_embedding_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/44182/jobs/named/sentence_embedding_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7a015538e4a0>, None)

In [15]:
# Updating the Feature Descriptions
# This step is for providing metadata information about our features

feature_descriptions = [
    {"name": "sentence", "description": "The actual sentences for which embeddings were created"},
    {"name": "embeddding", "description": "The embeddings for the sentences"},
    {"name": "p_key", "description": "This feature is used as a primary key"},
]

for desciption in feature_descriptions:
    sentence_embedding_fg.update_feature_description(desciption["name"], desciption["description"])

In [16]:
import hsfs

In [17]:
# Connecting to hopsworks feature stores and reading the data from the feature group which we had created earlier
connection = hsfs.connection()
fs = connection.get_feature_store(name='categoricalbins_featurestore')
fg = fs.get_feature_group('sentence_embedding', version=1)

Connected. Call `.close()` to terminate connection gracefully.


In [21]:
# Reading the feature group into a dataframe

df = fg.read()
df.head(2)

,sentence,embeddding,p_key
0,Hydrogen is the chemical element with the symb...,"[0.013064535,0.05343805,-0.02423306,0.04474186...",1
1,\nIt can be burned to produce heat or combined...,"[-0.020596532,0.112555,-0.03809536,0.08016463,...",21
